# 02. 파이썬을 이용한 자연어처리 기초

* 발표자 : 김무성

# 차례
* 텍스트 파일 핸들링
* 파이썬으로 영어와 한국어 텍스트 다루기
    - A. 자연어처리를 위한 파이썬 패키지 준비 & Load text
    - B. Text exploration
    - C. 형태소 분석

# 텍스트 파일 핸들링

### 1. 02_data/signal.txt 파일을 읽어서 출력하기

In [3]:
%ls 02_data/

signal.txt


In [97]:
fn = '02_data/signal.txt'
lines = []
with open(fn) as f :
    for line in f : 
        lines.append(line.strip())
        
print(lines[:10])

['Trying to let you know', 'Sign을 보내 signal 보내', 'I must let you know', 'Sign을 보내 signal 보내', 'Sign을 보내 signal 보내', 'Sign을 보내 signal 보내', 'Sign을 보내 signal 보내', 'I must let you know', 'Sign을 보내 signal 보내', '근데 전혀 안 통해']


------------------------

# 파이썬으로 영어와 한국어 텍스트 다루기

#### 참고자료 [1] 파이썬으로 영어와 한국어 텍스트 다루기 

#### Text analysis process

<img src="https://www.lucypark.kr/courses/2015-dm/images/text-process.png"  width= 300 />

# 전처리는 아래의 세부 과정으로 다시 한 번 나뉜다. 
1. Load text
2. Tokenize text (ex: stemming, morph analyzing)
3. Tag tokens (ex: POS, NER)
4. Token(Feature) selection and/or filter/rank tokens (ex: stopword removal, TF-IDF)
5. ...and so on (ex: calculate word/document similarities, cluster documents)


## A. 자연어처리를 위한 파이썬 패키지 준비 & Load text

### 1) 영어 자연어처리를 위한 nltk 셋팅

In [8]:
# Text corpora 다운로드 (아래의 두 데이터 다운로드하자)
# gutenberg
# maxent_treebank_pos_tagger

In [5]:
import nltk

nltk.download('gutenberg')
nltk.download('maxent_treebank_pos_tagger')

[nltk_data] Downloading package gutenberg to /home/jovyan/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


True

### 2) 한글 자연어처리를 위한 konlpy 셋팅

In [6]:
import konlpy

### 3) 토픽 모델링을 위한 gensim 셋팅

In [7]:
import gensim

## B. Text exploration

### 1) Read document

* 실습은 이 데이터들을 사용한다.
    - 영어: Jane Austen의 소설 Emma
    - 한국어 1: 대한민국 국회 제 1809890호 의안
    - 한국어 2: 트와이스의 시그널 가사

In [8]:
# 영어 데이터 로딩
# files_en = austen-emma.txt
# doc_en

In [9]:
from nltk.corpus import gutenberg   # Docs from project gutenberg.org
files_en = gutenberg.fileids()      # Get file ids

In [10]:
files_en

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [11]:
doc_en = gutenberg.open('austen-emma.txt').read()

In [12]:
doc_en[:100]

'[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a'

In [13]:
# 한국어 1 데이터 로딩
# files_ko_pol = "1809890.txt"
# doc_ko_pol

In [14]:
from konlpy.corpus import kobill    # Docs from pokr.kr/bill
files_ko_pol = kobill.fileids()         # Get file ids

In [15]:
files_ko_pol

['1809895.txt',
 '1809899.txt',
 '1809896.txt',
 '1809897.txt',
 '1809892.txt',
 '1809890.txt',
 '1809894.txt',
 '1809893.txt',
 '1809891.txt',
 '1809898.txt']

In [16]:
doc_ko_pol = kobill.open('1809890.txt').read()

In [17]:
doc_ko_pol[:100]

'지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.  11.  12.  \n\n발  의  자 : 정의화․이명수․김을동 \n\n이'

In [18]:
# 한국어 2 데이터 로딩
# files_ko_sig = '02_data/signal.txt'
# doc_ko_sig

In [19]:
files_ko_sig = '02_data/signal.txt'
with open(files_ko_sig) as f :
    doc_ko_sig = f.read()

In [20]:
doc_ko_sig[:100]

'Trying to let you know\nSign을 보내 signal 보내\nI must let you know\nSign을 보내 signal 보내\nSign을 보내 signal 보내\n'

### 2) Tokenize

문서를 토큰으로 나누는 방법은 다양하다. 여기서는 영어에는 nltk.tokenize.RegexpTokenizer, 한국어에는 konlpy.tag.Kkma.morph를 사용해보자.

In [21]:
# 영어 

In [22]:
from nltk.tokenize import RegexpTokenizer
t = RegexpTokenizer("[\w]+")
tokens_en = t.tokenize(doc_en)

In [23]:
tokens_en[:10]

['Emma', 'by', 'Jane', 'Austen', '1816', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma']

In [24]:
# 한글 1

In [25]:
from konlpy.tag import Kkma; 

In [26]:
kkma = Kkma()

In [27]:
tokens_ko_pol = kkma.morphs(doc_ko_pol)

In [28]:
tokens_ko_pol[:10]

['지방', '공무원', '법', '일부', '개정', '법률안', '(', '정의', '화', '의원']

In [29]:
# 한글 2

In [30]:
tokens_ko_sig = kkma.morphs(doc_ko_sig)

In [31]:
tokens_ko_sig[:10]

['Trying', 'to', 'let', 'you', 'know', 'Sign', '을', '보', '내', 'signal']

### 3) Load tokens with nltk.Text()

nltk.Text()는 문서 하나를 편리하게 탐색할 수 있는 다양한 기능을 제공한다.
* nltk.Text()
* Tokens
* Count
* Concordance
* Find similar words

#### nltk.Text() 

In [32]:
# 영어

In [33]:
en = nltk.Text(tokens_en)

In [34]:
en

<Text: Emma by Jane Austen 1816 VOLUME I CHAPTER...>

In [35]:
# 한글 1

In [36]:
ko_pol = nltk.Text(tokens_ko_pol, name='대한민국 국회 의안 제 1809890호')

In [37]:
ko_pol

<Text: 대한민국 국회 의안 제 1809890호>

In [38]:
# 한글 2

In [39]:
ko_sig = nltk.Text(tokens_ko_sig, name='트와이스의 시그널')

In [40]:
ko_sig

<Text: 트와이스의 시그널>

#### Tokens

In [41]:
# 영어

In [50]:
print(len(en.tokens))       # returns number of tokens (document length)
print(len(set(en.tokens)))  # returns number of unique tokens
en.vocab().elements                  # returns frequency distribution

161983
7723


<bound method Counter.elements of FreqDist({'to': 5183, 'the': 4844, 'and': 4672, 'of': 4279, 'I': 3178, 'a': 3004, 'was': 2385, 'her': 2381, 'it': 2128, 'in': 2118, ...})>

In [51]:
# 한글 1

In [52]:
print(len(ko_pol.tokens))       # returns number of tokens (document length)
print(len(set(ko_pol.tokens)))  # returns number of unique tokens
ko_pol.vocab().elements                  # returns frequency distribution

2033
448


<bound method Counter.elements of FreqDist({'--': 123, '하': 71, '의': 57, '.': 53, '이': 42, '육아': 39, '는': 38, 'ㄴ': 37, '을': 35, '휴직': 31, ...})>

In [53]:
# 한글 2

In [54]:
print(len(ko_sig.tokens))       # returns number of tokens (document length)
print(len(set(ko_sig.tokens)))  # returns number of unique tokens
ko_sig.vocab().elements                  # returns frequency distribution

458
129


<bound method Counter.elements of FreqDist({'내': 32, '보': 30, '찌릿': 24, '을': 21, 'signal': 13, 'Sign': 11, '어': 10, '왜': 9, '게': 9, '네': 8, ...})>

#### Count

In [55]:
# 영어

In [56]:
en.count('Emma')        # Counts occurrences

865

In [57]:
# 한글 1

In [58]:
ko_pol.count('초등학교')   # Counts occurrences

6

In [59]:
# 한글 2

In [60]:
ko_sig.count('signal')   # Counts occurrences

13

#### Concordance

In [61]:
# 영어

In [62]:
en.concordance('Emma', lines=5)

Displaying 5 of 865 matches:
                                     Emma by Jane Austen 1816 VOLUME I CHAPTER
                                     Emma Woodhouse handsome clever and rich w
f both daughters but particularly of Emma Between _them_ it was more the intim
nd friend very mutually attached and Emma doing just what she liked highly est
 by her own The real evils indeed of Emma s situation were the power of having


In [63]:
# 한글 1

In [64]:
ko_pol.concordance('초등학교')

Displaying 6 of 6 matches:
김 정훈 김 학 송 의원 ( 10 인 ) 제안 이유 및 주요 내용 초등학교 저학년 의 경우 에 도 부모 의 따뜻 하 ㄴ 사랑과 보살핌 이 필
ㄹ 수 있 는 자녀 의 나이 는 만 6 세 이하 로 되 어 있 어 초등학교 저학년 이 ㄴ 자녀 를 돌보 기 위하 어서 는 해당 부모님 은 일
 저 의 63 조 제 2 항 제 4 호 중 “ 만 6 세 이하 의 초등학교 취학 전 자녀 를 ” 을 “ 만 8 세 이하 ( 취학 중 이 ㄴ 
녀 를 ” 을 “ 만 8 세 이하 ( 취학 중 이 ㄴ 경우 에 는 초등학교 2 학년 이하 를 말하 ㄴ다 ) 의 자녀 를 ” 로 하 ㄴ다 . 
 ∼ 3 . ( 현행 과 같 음 ) 4 . 마 ㄴ 6 세 이하 의 초등학교 취 4 . 마 ㄴ 8 세 이하 ( 취학 중 이 ㄴ 경우 학 전 자
( 취학 중 이 ㄴ 경우 학 전 자녀 를 양육 하 기 위하 에 는 초등학교 2 학년 이하 를 여 필요 하 거나 여자 공무원 이 말하 ㄴ다 )


In [65]:
# 한글 2

In [66]:
ko_sig.concordance('signal')

Displaying 15 of 15 matches:
                                    signal 보 내 I must let you know Sign 을 보 내 
 보 내 I must let you know Sign 을 보 내 signal 보 내 Sign 을 보 내 signal 보 내 Sign 을 보 
ow Sign 을 보 내 signal 보 내 Sign 을 보 내 signal 보 내 Sign 을 보 내 signal 보 내 Sign 을 보 
 내 Sign 을 보 내 signal 보 내 Sign 을 보 내 signal 보 내 Sign 을 보 내 signal 보 내 I must le
 내 Sign 을 보 내 signal 보 내 Sign 을 보 내 signal 보 내 I must let you know Sign 을 보 내 
 보 내 I must let you know Sign 을 보 내 signal 보 내 근데 전혀 알 ㄴ 통해 눈빛 을 보 내 눈치 를 주 네 
 모르 겠 다 정말 다시 한 번 힘 을 내서 Sign 을 보 내 signal 보 내 눈짓 도 손짓 도 어떻 ㄴ 표정 도 소용 이 없 네 하나
ㄹ 친구 로 만 대하 ㄹ래 내 가 원하 는 건 그 게 아니 ㄴ데 Signal 보 내 signal 보 내 찌릿 찌릿 찌릿 찌릿 나 는 너 를 
 ㄹ래 내 가 원하 는 건 그 게 아니 ㄴ데 Signal 보 내 signal 보 내 찌릿 찌릿 찌릿 찌릿 나 는 너 를 원하 어 나 는 너 
니 Trying to let you know Sign 을 보 내 signal 보 내 I must let you know Sign 을 보 내 
 보 내 I must let you know Sign 을 보 내 signal 보 내 널 보 며 웃 으면 알아채 야지 오늘 만 몇 번째 널 보
ㄹ 친구 로 만 대하 ㄹ래 내 가 원하 는 건 그 게 아니 ㄴ데 Signal 보 내 signal 보 내 찌릿 찌릿 찌릿 찌릿 나 는 너 를 
 ㄹ래 내 가 원하 는 건 그 게 아니 ㄴ

#### Find similar words

In [67]:
# 영어

In [68]:
en.similar('Emma')
en.similar('Frank')

she it he i harriet you her jane him that me all and they them herself
there say hartfield isabella
mrs mr emma harriet it you she her him hartfield he them isabella that
jane all herself i highbury mind


In [69]:
# 한글 1

In [70]:
ko_pol.similar('자녀')
ko_pol.similar('육아휴직')

논의
No matches


In [71]:
# 한글 2

In [72]:
ko_sig.similar('signal')
ko_sig.similar('Sign')

널
맘


#### Collocations

In [73]:
# stopwords를 다운받아야 한다.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
# 영어

In [75]:
en.collocations()

Mrs Weston; Frank Churchill; Miss Woodhouse; Mrs Elton; Miss Bates;
Jane Fairfax; Miss Fairfax; every thing; young man; every body; great
deal; Mrs Goddard; dare say; Maple Grove; John Knightley; Miss Taylor;
Miss Smith; Robert Martin; Colonel Campbell; Box Hill


In [76]:
# 한글 1

In [77]:
ko_pol.collocations()

초등학교 저학년


In [78]:
# 한글 2

In [79]:
ko_sig.collocations()

know Sign; must let


## C. 형태소 분석

In [80]:
# 영어 

In [81]:
tokens_en[:10]

['Emma', 'by', 'Jane', 'Austen', '1816', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma']

In [82]:
# 어간 추출(stemming)은 여러가지 이유로 변화된 단어의 접미사나 어미를 제거하여 
# 같은 의미를 가지는 형태소의 실제 형태를 동일하게 만드는 방법이다. 
# NLTK는 PorterStemmer LancasterStemmer 등을 제공한다. 
from nltk.stem import PorterStemmer
st = PorterStemmer()
[st.stem(w) for w in tokens_en][:10]

['emma', 'by', 'jane', 'austen', '1816', 'volum', 'I', 'chapter', 'I', 'emma']

In [83]:
# 어간 추출은 원형 복원(lemmatizing)의 일종이다. 
# 원형 복원은 같은 의미를 가지는 여러 단어를 가장 근본적인 형태 즉 사전형으로 통일하는 작업이다.

# WordNetLemmatizer 사용을 위해 필요함
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [84]:
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
[lm.lemmatize(w) for w in tokens_en[:10]]

['Emma', 'by', 'Jane', 'Austen', '1816', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma']

In [85]:
# 품사(POS, part-of-speech)는 낱말을 문법적인 기능이나 형태, 뜻에 따라 구분한 것이다.

# averaged_perceptron_tagger를 사용하기 위해.
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [86]:
tags_en = nltk.pos_tag(tokens_en[:10])

In [87]:
tags_en

[('Emma', 'NN'),
 ('by', 'IN'),
 ('Jane', 'NNP'),
 ('Austen', 'NNP'),
 ('1816', 'CD'),
 ('VOLUME', 'NNP'),
 ('I', 'PRP'),
 ('CHAPTER', 'VBP'),
 ('I', 'PRP'),
 ('Emma', 'VBP')]

In [88]:
# 한글 1 
#예) tags = kkma.pos("작고 노란 강아지가 페르시안 고양이에게 짖었다")

In [89]:
tags = kkma.pos("작고 노란 강아지가 페르시안 고양이에게 짖었다")
tags

[('작고', 'NNG'),
 ('노', 'NNG'),
 ('이', 'VCP'),
 ('란', 'ETD'),
 ('강아지', 'NNG'),
 ('가', 'JKS'),
 ('페르', 'NNG'),
 ('시안', 'NNP'),
 ('고양이', 'NNG'),
 ('에게', 'JKM'),
 ('짖', 'VV'),
 ('었', 'EPT'),
 ('다', 'EFN')]

In [90]:
tokens_ko_pol[:10]

['지방', '공무원', '법', '일부', '개정', '법률안', '(', '정의', '화', '의원']

In [91]:
tags_ko_pol = [kkma.pos(token)[0] for token in tokens_ko_pol[:10]] 

In [92]:
tags_ko_pol

[('지방', 'NNG'),
 ('공무원', 'NNG'),
 ('법', 'NNG'),
 ('일부', 'NNG'),
 ('개정', 'NNG'),
 ('법률안', 'NNG'),
 ('(', 'SS'),
 ('정의', 'NNG'),
 ('화', 'NNG'),
 ('의원', 'NNG')]

In [93]:
# 한글 2

In [94]:
tokens_ko_sig[60:70]

['ㄴ', '통해', '눈빛', '을', '보', '내', '눈치', '를', '주', '네']

In [95]:
tags_ko_sig = [kkma.pos(token)[0] for token in tokens_ko_sig[60:70]] 

In [96]:
tags_ko_sig

[('ㄴ', 'NNG'),
 ('통하', 'VV'),
 ('눈빛', 'NNG'),
 ('을', 'NNG'),
 ('보', 'NNG'),
 ('내', 'NNG'),
 ('눈치', 'NNG'),
 ('를', 'UN'),
 ('주', 'NNG'),
 ('네', 'MDN')]

# 참고자료 
* [1] 파이썬으로 영어와 한국어 텍스트 다루기 - https://www.lucypark.kr/courses/2015-dm/text-mining.html
* [2] NLTK 자연어 처리 패키지 - https://datascienceschool.net/view-notebook/118731eec74b4ad3bdd2f89bab077e1b/ 